In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.simplefilter("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from pylab import rcParams
sns.set_style("whitegrid", {'axes.grid' : True})

plt.rc('figure', figsize=(10, 6))

from plotly.offline import init_notebook_mode, iplot
import plotly
import plotly.graph_objs as go
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', None)

from sklearn.impute import SimpleImputer
from scipy.stats import kurtosis, kurtosistest
from scipy.stats import skew, skewtest
from scipy.stats import ttest_ind
from scipy.stats import f_oneway
from scipy.stats import spearmanr
from scipy.stats.stats import pearsonr
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [2]:
#parsing_day_april = '2022-12-28'
parsing_day_april = '2023-05-07'
#parsing_day_soc = '2023-01-18'
parsing_day_soc = '2023-06-01'
#parsing_day_zdor = '2023-01-16'
parsing_day_zdor = '2023-05-30'
# Словарь для организации файлов
dict_num_pars_day = {}

with open(fr'C:\Users\user\Scrapy_project\CSV_file\dict_num_pars_day.txt', 'r', encoding='utf-8') as fn:
    list_file = fn.read()
#     print(list_file.splitlines()[1:])
    for i in list_file.splitlines()[1:]:
        i = i.split(';')
        l = [int(i[1].strip()), i[2].strip()]
        dict_num_pars_day[i[0].strip()] = l#.append(f'{i[1], i[2]}')
#         print(f'{i[0]}, {i[1]}, {i[2]}')
print(dict_num_pars_day)
print(f'{parsing_day_april[:4]}_{parsing_day_april[5:7]}_{parsing_day_april[8:10]}')

{'2022-12-28': [1, 'apr'], '2023-05-07': [2, 'apr'], '2023-01-18': [1, 'soc'], '2023-06-01': [2, 'soc'], '2023-01-16': [1, 'zdor'], '2023-05-30': [2, 'zdor']}
2023_05_07


In [3]:
# Загружаем файлы CSV в датафрейм каждой отдельно сети
df_soc = pd.read_csv(fr'C:\Users\user\Scrapy_project\CSV_file\csv_SOCIALAPT_href_{parsing_day_soc}.csv', sep=',')#?raw=true
df_zdor = pd.read_csv(fr'C:\Users\user\Scrapy_project\CSV_file\csv_NAZDOROV_href_{parsing_day_zdor}.csv', sep=',')#?raw=true
df_apr = pd.read_csv(fr'C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\csv_APRIL_page_parsing_index_{parsing_day_april}.csv', sep=',')#?raw=true
df_apr_prop = pd.read_csv(fr'C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\csv_properties_APRIL_index_{parsing_day_april}.csv', sep=',')#?raw=true
df_apr_price = pd.read_csv(fr'C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\csv_price_APRIL_index_{parsing_day_april}.csv', sep=',')#?raw=true
# файд товарных групп group_goods_april формируется в момент работы BaeutifulSoup при формировании csv файла
df_group_goods = pd.read_csv(fr'C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\group_goods_april.csv', sep=',')#?raw=true
keys = df_group_goods['group'].to_list()
vals = df_group_goods['group2'].to_list()
dict_group_goods= {k: v for k, v in zip(keys, vals)}

In [4]:
# ЗДЕСЬ МОЖНО ОЦЕНИТЬ, НАПЕЧАТАТЬ ДУПЛИКАТЫ В 3-Х ФАЙЛАХ СЕТИ АПРЕЛЬ.
xx = df_apr.duplicated(subset= ['name', 'ID', 'properties'], keep= 'first').value_counts()
yy = df_apr_prop.duplicated(subset= ['Состав, вес (кратко)', 'Производитель', 
                           'Страна', 'Бренд', 'name_item_prop'], keep= 'first').value_counts()
zz = df_apr_price.duplicated(subset= ['name_item_price'], keep= 'first').value_counts()
# print('apr', xx)
# print('prop', yy)
# print('price', zz)
# df_apr.duplicated(subset= ['name', 'ID', 'group'], keep= 'first').value_counts()

In [5]:
# ЗДЕСЬ СМОТРИМ РАЗМЕР ТРЁХ БАЗ СЕТИ АПРЕЛЬ
apr_total = df_apr.shape
apr_prop = df_apr_prop.shape
apr_price = df_apr_price.shape
print(apr_total[0], apr_prop[0], apr_price[0])
# ЗАНОСИМ БАЗУ В ФАЙЛ CSV
df_apr_prep = pd.merge(df_apr, df_apr_prop, how='left', left_on='count_index', right_on='count_index')
df_apr_total_prep = pd.merge(df_apr_prep, df_apr_price, how='left', left_on='count_index', right_on='count_index')

df_apr_total_prep.to_csv(fr'C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\df_apr_total.csv', sep=',', index = False)

2322 2322 2322


In [6]:
# df_apr_total_prep.duplicated(subset= ['name_item_price', 'Состав, вес (кратко)', 'Производитель', 
#                            'Страна', 'Бренд', 'name_item_prop', 'name', 'ID'], keep= 'first').value_counts()

In [7]:
# УБИРАЕМ ДУПЛИКАТЫ В БАЗЕ СЕТИ АПРЕЛЬ
df_apr_total = df_apr_total_prep[['count_index', 'allowOnlinePayment', 'isInStock', 'categoryID', 'ID',
       'slug', 'bonuses', 'name', 'averageRating', 'limitWithoutCard', 'isAvailable', 'allowDelivery', 
       'rating', 'isRecipe', 'group', 'parsing_day_x', 'Нозология', 'Действующее вещество', 
       'Группа товара', 'Бренд', 'Производитель', 'Страна', 'Форма выпуска', 'Рецепт', 
       'Условия хранения', 'Действующее вещество, сборное название', 'Состав, вес (кратко)', 
       'Фармакотерапевтическая группа', 'price_withCard', 'price_withPeriod', 'price_withoutCard']]
df_apr_total.sample(3)
df_apr_total['Фармакотерапевтическая группа'].value_counts()
df_apr_total.duplicated(subset= ['Состав, вес (кратко)', 'Производитель', 
                           'Страна', 'Бренд', 'name', 'ID'], keep= 'first').value_counts()
df_apr_total.drop_duplicates(subset= ['Состав, вес (кратко)', 'Производитель', 
                           'Страна', 'Бренд', 'name', 'ID'], keep= 'first', inplace= True)
df_apr_total.duplicated(subset= ['Состав, вес (кратко)', 'Производитель', 
                           'Страна', 'Бренд', 'name', 'ID'], keep= 'first').value_counts()

False    2272
dtype: int64

In [8]:
# В КОЛОНКАХ МЕНЯЕМ РАЗНЫЕ СИМВОЛЫ И ПЕРЕДЕЛЫВАЕМ В ТИП ДАННЫХ СТРОКА
# В ФАРМАКОТЕРАПЕВТИЧЕСКОЙ ГРУППЕ СЧИТАЕМ КОЛ-ВО ТОВАРОВ И ДОЛЮ В ОБЩЕМ КОЛ-ВЕ ПО ГРУППЕ
df_apr_total.columns
columns_total = ['Нозология', 'Действующее вещество', 'Группа товара', 'Бренд',
       'Производитель', 'Страна', 'Форма выпуска', 'Рецепт',
       'Условия хранения', 'Действующее вещество, сборное название',
       'Состав, вес (кратко)', 'Фармакотерапевтическая группа']

value_replace = {'[': '', ']': '', "'": ''}

df_apr_total['Нозология'] = df_apr_total['Нозология'].apply(lambda x: x.replace('[', ''))
for i in columns_total:
    df_apr_total[i] = df_apr_total[i].apply(lambda x: x.replace('[]', 'Not value'))
    df_apr_total[i] = df_apr_total[i].apply(lambda x: x.replace('[', ''))
    df_apr_total[i] = df_apr_total[i].apply(lambda x: x.replace(']', ''))
    df_apr_total[i] = df_apr_total[i].apply(lambda x: x.replace("'", ''))
    df_apr_total.head(3)
# df_apr_total['Группа товара'].value_counts().to_dict()
# count = 0
# for k, v in df_apr_total.loc[df_apr_total['Фармакотерапевтическая группа'] != 'Not value']['Фармакотерапевтическая группа'].value_counts().to_dict().items():
#     count += v
#     share_total = df_apr_total.loc[df_apr_total['Фармакотерапевтическая группа'] == 'Not value']['Фармакотерапевтическая группа'].count()
#     share = count / (df_apr_total.shape[0] - share_total)
#     print(f'{k}, {v}, ====== > {count}, ----- > {round(share*100, 2)}%')
#     if share <= 0.805:
#         print(f'{k}, {v}, ====== > {count}, ----- > {round(share*100, 2)}%')
# df_apr_total.shape[0]

In [9]:
# МЕНЯЕМ False и True на 0 и 1
df_apr_total.loc[df_apr_total['Фармакотерапевтическая группа'] == 'Средства от боли']['Форма выпуска'].value_counts()
df_apr_total.loc[df_apr_total['Форма выпуска'] == 'Таблетки']#.value_counts()
false_true = {False: 0, True: 1}
group_change = {'sredstva': 'sredstva_dlya_lecheniya_prostudy_i_grippa'}
columns_map =  ['allowOnlinePayment', 'isInStock', 'isAvailable', 'allowDelivery', 'isRecipe']
for i in columns_map:
    df_apr_total[i]= df_apr_total[i].map(false_true)
# df_apr_total['group']= df_apr_total['group'].map(group_change)

In [10]:
# МЕНЯЕМ КИРИЛЛИЦУ НА ЛАТИННИЦУ В НАЗВАНИИ КОЛОНОК
rename_columns1 = {'Нозология': 'Nosology', 'Действующее вещество': 'Active_ingredient', 
'Группа товара': 'Product_Group', 'Бренд': 'Brand', 'Производитель': 'Manufacturer', 
'Страна': 'Country', 'Форма выпуска': 'Form_release', 'Рецепт': 'Prescription', 
'Условия хранения': 'Conditions_of_storage', 'slug': 'name_latin', 
'Действующее вещество, сборное название': 'Active_ingredient_compound_name_composition', 
'Состав, вес (кратко)': 'weight_briefly', 'Фармакотерапевтическая группа': 'Pharmacotherapeutic_group'} 
list_col = df_apr_total.columns
list_col
df_apr_total.rename(columns=rename_columns1, inplace=True)
# МЕНЯЕМ НАИМЕНОВАНИЯ БОЛЬШИХ ГРУПП ТОВАРА НА НУЖНЫЕ, ЧТОБЫ ИСПРАВИТЬ ОШИБКИ ПРИ ПАРСИНГЕ
list_col = df_apr_total.columns
list_col
group_change = {'serdechnososudistaya': 'serdechnososudistaya', 'obezbolivayushchie': 'obezbolivayushchie', 
'sredstva': 'sredstva_dlya_lecheniya_prostudy_i_grippa', 'zheludochnokishechnye': 'zheludochnokishechnye'}
df_apr_total['bonuses'] = df_apr_total['bonuses'].fillna(0)
df_apr_total['group']= df_apr_total['group'].map(group_change)
# df_apr_total.sample(4)
# df_apr_total.loc[300:302]

In [11]:
# ПОСМОТРЕТЬ СПИСОК УНИКАЛЬНЫХ ЗНАЧЕНИЙ ВО ВСЕХ КОЛОНКАХ. ПОКА МОЖНО СКРЫТЬ, ПРИ НЕОБХОДИМОСТИ ПОСМОТРЕТЬ И 
# ТАКЖЕ НУЛЕВЫЕ ЗНАЧЕНИЯ ВО ВТОРОЙ ЧАСТИ
# for i in list_col:
#     x = df_apr_total[i].unique()
#     if len(x) <=100:
#         print('\033[1m'+i+'\033[0;0m', '\033[1m'+f'Уникальных значений: {len(x)}'+'\033[0;0m', x, sep='\n')
#         print('--------------------------------------------------------------------')
#     else:
#         print('\033[1m'+i+'\033[0;0m', '\033[1m'+f'Уникальных значений: {len(x)}'+'\033[0;0m'+f' more 100', sep='\n')
#         print('--------------------------------------------------------------------')
#     x = df_apr_total[i].isna().sum()
#     if x != 0:
#         print('\033[1m'+i+'\033[0;0m', '\033[1m'+f'Значений NaN :'+'\033[0;0m', x, sep='\n')
#         print('--------------------------------------------------------------------')

In [12]:
# ДОБАВЛЯЕМ КОЛОНКУ С ФОРМОЙ ВЫПУСКА, КОТОРАЯ БУДЕТ БОЛЕЕ ОБЩАЯ
df_apr_total.group.value_counts()
# df_apr_total.loc[df_apr_total['group'].isna()]#.value_counts()
# df_apr_total.describe()
# df_apr_total.columns
# df_apr.loc[300]
columns_new = ['count_index', 'name', 'price_withCard', 'price_withPeriod', 'price_withoutCard',
               'allowOnlinePayment', 'isInStock', 'name_latin', 'bonuses', 'averageRating', 
               'isAvailable', 'allowDelivery', 'isRecipe', 'group', 'parsing_day_x', 
               'Nosology', 'Active_ingredient', 'Product_Group', 'Brand', 'Manufacturer', 'Country', 
               'Form_release', 'Form_release_common', 'Prescription', 'Conditions_of_storage', 
               'Active_ingredient_compound_name_composition', 'weight_briefly', 'Pharmacotherapeutic_group']

form_release_change = {'Таблетки': 'Таблетки', 'Капсулы для приема внутрь': 'Капсулы', 'Раствор для инъекций': 'Растворы', 
'Порошок для приготовления раствора для внутреннего применения': 'Порошок', 'Таблетки для рассасывания': 'Таблетки', 
'Суппозитории ректальные': 'Суппозитории', 'Сироп для приема внутрь': 'Сироп', 'Спрей назальный': 'Спрей', 
'Гель для наружного применения': 'Гель', 'Суспензия для приема внутрь': 'Суспензия', 'Капли для приема внутрь': 'Капли', 
'Спрей для местного применения': 'Спрей', 'Мазь для наружного применения': 'Мазь', 'Раствор для приема внутрь': 'Растворы', 
'Таблетки жевательные': 'Таблетки', 'Капли назальные': 'Капли', 
'Лиофилизат для приготовления раствора для инъекций': 'Лиофилизат', 'Пастилки': 'Пастилки', 'Гранулы': 'Гранулы', 
'Таблетки шипучие': 'Таблетки', 'Раствор для местного применения': 'Растворы', 'Крем для наружного применения': 'Крем', 
'Порошок для приготовления суспензии для приема внутрь': 'Порошок', 'Лейкопластырь медицинский лечебный': 'Пластырь', 
'Таблетки диспергируемые': 'Таблетки', 'Гель для приема внутрь': 'Гель', 'Масло для приема внутрь': 'Масло', 
'Настойка': 'Настойка', 'Аэрозоль для наружного применения': 'Аэрозоль', 'Бальзам для наружного применения': 'Бальзам', 
'Капли ушные': 'Капли', 'Раствор для приема внутрь и ингаляций': 'Растворы', 'Масло для наружного применения': 'Масло', 
'Not value': 'Not value', 'Порошок': 'Порошок', 'Набор таблеток и капсулы': 'Таблетки', 'Плоды порошок': 'Плоды порошок', 
'Драже': 'Драже', 'Мазь назальная': 'Мазь', 'Таблетки растворимые': 'Таблетки', 
'Спрей дозированный для сублингвального применения': 'Спрей', 'Порошок для наружного применения': 'Порошок', 
'Карандаш для ингаляций': 'Карандаш', 'Раствор для ректального введения': 'Растворы', 'Саше': 'Саше', 
'Суппозитории ректальные и вагинальные': 'Суппозитории', 'Раствор для инфузий': 'Растворы', 
'Лиофилизат для приготовления раствора для инъекций и местного применения': 'Лиофилизат', 'Эмульсия': 'Эмульсия', 
'Плоды цельные': 'Плоды цельные', 'Раствор для наружного применения': 'Растворы', 
'Лиофилизат для приготовления раствора для инфузий': 'Лиофилизат', 'Пастилки жевательные': 'Пастилки', 
'Раствор для наружного применения и ингаляций': 'Растворы', 'Суппозитории вагинальные': 'Суппозитории', 
'Лиофилизат для приготовления назального раствора': 'Лиофилизат', 'Пластырь': 'Пластырь', 
'Раствор для приема внутрь, местного и наружного применения': 
'Растворы', 'Таблетки для приготовления раствора для местного применения': 'Таблетки', 'Сок': 'Сок', 
'Гель для ректального применения': 'Гель', 'Концентрат для приготовления раствора для инфузий': 'Концентрат', 
'Капсулы подъязычные': 'Капсулы', 'Порошок для приготовления раствора для инфузий': 'Порошок'}

df_apr_total.insert (loc= 23 , column='Form_release_common', value=df_apr_total.Form_release)
df_apr_total['Form_release_common']= df_apr_total['Form_release_common'].map(form_release_change)
df_apr_total_pars_day = df_apr_total[columns_new]
# ДЕЛАЕМ ДАТАФРЕЙМ С НЕНУЛЕВОЙ ЦЕНОЙ И ЗАПИСЫВАЕМ ЕГО В CSV ФАЙЛ 
df_apr_total_pars_day = df_apr_total_pars_day.loc[df_apr_total['price_withCard'] != 0]#['price_withCard'].value_counts()
df_apr_total_pars_day.to_csv(fr'C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\df_apr_total_{parsing_day_april[:4]}_{parsing_day_april[5:7]}_{parsing_day_april[8:10]}.csv',
                               sep=',', encoding= 'utf-8', index= False)


In [13]:
# ДЕЛАЕМ ФУНКЦИЮ ДЛЯ ПЕЧАТИ УНИКАЛЬНЫХ ЗНАЧЕНИЙ. ПОКА МОЖНО ЗАКОММЕНТИРОВАТЬ ПОСЛЕДНЮЮ 
df_apr_total.Form_release_common.value_counts()
def list_unique(df, d= 1):
    list_col = df.columns
    if d == 1:
        for i in list_col:
            x = df[i].unique()
            if len(x) <=100:
                print('\033[1m'+i+'\033[0;0m', '\033[1m'+f'Уникальных значений: {len(x)}'+'\033[0;0m', x, sep='\n')
                print('--------------------------------------------------------------------')
            else:
                print('\033[1m'+i+'\033[0;0m', '\033[1m'+f'Уникальных значений: {len(x)}'+'\033[0;0m'+f' more 100', sep='\n')
                print('--------------------------------------------------------------------')
    else:
        for i in list_col:
            x = df[i].isna().sum()
            if x != 0:
                print('\033[1m'+i+'\033[0;0m', '\033[1m'+f'Значений NaN :'+'\033[0;0m', x, sep='\n')
                print('--------------------------------------------------------------------')
#y = list_unique(df_apr_total_pars_day)

In [14]:
# ЗАПИВЫВАЕМ В ДАТАФРЕЙМ ИЗ CSV ФАЙЛА, В КОТОРОМ ЗАПИСАНЫ РАЗНЫЕ ДАТЫ ПАРСИНГА
#df_apr_prop = pd.read_csv(fr'C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\csv_properties_APRIL_index_{parsing_day_april}.csv', sep=',')
df_apr_prep = pd.read_csv(fr'C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\df_apr_total_{parsing_day_april[:4]}_{parsing_day_april[5:7]}_{parsing_day_april[8:10]}.csv', sep=',')
#df_conect_2023_05_07 = pd.read_csv(fr'C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\df_apr_total_2023_05_07.csv', sep=',')

# df_conect_2022_12_28.sample(4)
# df_conect_2023_05_07.sample(4)
# отменяем ОБЪЕДИНИТЬ БАЗЫ ИЗ РАЗНЫХ БАЗ В ОДНУ С ПОМОЩЬЮ concat, т.е. одна дата под другой.
#shape1 = df_conect_2022_12_28.shape
#shape2 = df_conect_2023_05_07.shape
#print(shape1, shape2)
#df_apr_prep = pd.concat([df_conect_2022_12_28, df_conect_2023_05_07], ignore_index=True)
#df_apr_prep.sample(4)

## Na zdorovie

In [15]:
df_zdor['Производитель'].value_counts().to_dict()
df_apr_prep.Manufacturer.value_counts().to_dict()
list_apr_manufact = list(df_apr_prep.Manufacturer.value_counts().to_dict().keys())
len(list_apr_manufact)


305

In [16]:
# ЭТА ФУНКЦИЯ ОСТАВЛЯЕТ ТОЛЬКО БУКВЫ В СТРОКАХ. В ДАННОМ СЛУЧАЕ В НАЗВАНИИ ПРОИЗВОДИТЕЛЕЙ УБРАТЬ 
# КАВЫЧКИ, ТОЧКИ И Т.П. ТАКЖЕ (ЗАО, ПАО...)
def parse_words(string_):
    lat = "a-zA-Z"
    cyr = "а-яА-ЯёЁ"
    all_words = re.findall(rf"\b([{lat}{cyr}]+)\b", string_)
    latin_words = re.findall(rf"\b([{lat}]+)\b", string_)
    cyrillic_words = re.findall(rf"\b([{cyr}]+)\b", string_)
    mixed_words = set(all_words) - (set(latin_words) | set(cyrillic_words))
#     return " ".join(latin_words), " ".join(cyrillic_words), " ".join(mixed_words)
    return tuple(map(" ".join, (latin_words, cyrillic_words, mixed_words)))
#     return tuple(map(''.join, (cyrillic_words)))
dict_apr_manufact = {}
for i in list_apr_manufact:
    c = i.count(f'/')
    a = parse_words(i)
    
    if (c == 0 or c == 1) and a[1]  == '':
#         print('I   ', c, '<=>', i, ' -------> ', a[0])#[1])
        dict_apr_manufact[i] = a[0]
    elif c > 1 and a[1]  == '':
#         print('II  ', c, '<=>', i, ' -------> ', a[0])#[1])
        dict_apr_manufact[i] = a[0]
    elif (c == 0 or c == 1) and a[1]  != '':
#         print('III ',  c, '<=>', i, ' -------> ', a[1])#[1])
        dict_apr_manufact[i] = a[1]
    elif c > 1 and a[1]  == '':
#         print('IV  ',  c, '<=>', i, ' -------> ', a[1])#[1])
        dict_apr_manufact[i] = a[1]
    elif c > 1 and a[1]  != '':
#         print('V  ',  c, '<=>', i, ' -------> ', a[1])
        dict_apr_manufact[i] = a[1]
#     if c > 1 and a[1]  != '':
#         print('V  ',  c, '<=>', i, ' -------> ', a[1])#[1])
print(len(dict_apr_manufact))
# print(dict_apr_manufact)
for k, v in dict_apr_manufact.items():
    v = v.replace('ЗАО', '').replace('ООО', '').replace('ПАО', '').replace('ОАО', '').replace('АО', '').replace('ПКФ', '').strip()
    v = v.replace('ПКФ', '').replace('ПФК', '').replace('ФП', '').strip()
    dict_apr_manufact[k] = v
#     print(k, '=', v)

# ПИШЕМ В ТЕКСТОВЫЙ ФАЙЛ ГРУППЫ ПРОИЗВОДИТЕЛЕЙ ОТ САЙТА АПРЕЛЬ
with open(fr"C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\dict_apr_manufact.txt", 'w') as file:
    for k, v in dict_apr_manufact.items():
        file.write(f'{k}; {v}\n')

305


In [17]:
# МЕНЯЕМ ТОЧКИ, КАВЫЧКИ И Т.П. В БАЗЕ НАЗДОРОВЬЕ
df_zdor['Производитель'].value_counts().to_dict()
df_zdor['Производитель'].value_counts().to_dict()
list_zdor_manufact = list(df_zdor['Производитель'].value_counts().to_dict().keys())
len(list_zdor_manufact)
dict_zdor_manufact = {}
count = 0
for i in list_zdor_manufact:
    c = i.count(f'/')
    a = parse_words(i)
#     print('#  ',  c, '<=>', i, ' -------> ', a)#[1])
    if c == 0:
#         print('I   ', c, '<=>', i, ' -------> ', a[1])#[1])
        dict_zdor_manufact[i] = a[1]
    elif c > 0:
        xxx = i.split('/')
#         print('II  ', c, '<=>', i, ' -------> ', xxx)
        dict_zdor_manufact[i] = xxx
#     elif (c == 0 or c == 1) and a[1]  != '':
#         print('III ',  c, '<=>', i, ' -------> ', a[1])#[1])
#         dict_zdor_manufact[i] = a[1]
#     elif c > 1 and a[1]  == '':
#         print('IV  ',  c, '<=>', i, ' -------> ', a[1])#[1])
#         dict_zdor_manufact[i] = a[1]
#     elif c > 1 and a[1]  != '':
#         print('V  ',  c, '<=>', i, ' -------> ', a[1])
#         dict_zdor_manufact[i] = a[1]
#     if c > 0:# a[0]  == '':
#         count += 1
#         xxx = i.split('/')#[1]#.split('.')[0]
#         print(f'# {count} ',  c, '<=>', i, ' -------> ', xxx)# if xxx == '' else ''#[1])
print(len(dict_zdor_manufact))
# print(dict_apr_manufact)
for k, v in dict_zdor_manufact.items():
    if type(v) is list:
        v1 = []
        for i in v:
            i = i.replace('НАО', '').replace('НПО', '').replace('пр.', '').replace('уп', '').strip()
            i = i.replace('ЗАО', '').replace('ООО', '').replace('ПАО', '').replace('ОАО', '').replace('АО', '').replace('ПКФ', '').strip()
            i = i.replace('ПКФ', '').replace('ПФК', '').replace('ФП', '').strip()
            v1.append(i)
        v=v1
        dict_zdor_manufact[k] = v
    else:
        v = v.replace('НАО', '').replace('НПО', '').strip()
        v = v.replace('ЗАО', '').replace('ООО', '').replace('ПАО', '').replace('ОАО', '').replace('АО', '').replace('ПКФ', '').strip()
        v = v.replace('ПКФ', '').replace('ПФК', '').replace('ФП', '').strip()
    dict_zdor_manufact[k] = v
#     print(k, '=', v)

with open(fr"C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\dict_zdor_manufact.txt", 'w') as file:
    for k, v in dict_zdor_manufact.items():
        file.write(f'{k}; {v}\n')

432


In [18]:
# def creat_group_goods() ФАЙЛ group_goods_all.txt ПРОИЗВОДИТЕЛИ. ФОРМИРУЕМ САМОСТОЯТЕЛЬНО ПУТЁМ ОБРАБОТКИ ФАЙЛА EXCEL РУЧНЫМ МАТЧИНГОМ
l_apr_keys, l_zdor_keys, l_soc_keys = [], [], []
l_apr_values, l_zdor_values, l_soc_values = [], [], []
with open(fr"C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\group_goods_all.txt", 'r', encoding='utf-8') as fp:
    list_file = fp.read()
    for i in list_file.splitlines():
        j = i.split('|')
        if j[3] == 'Апрель':
            l_apr_keys.append(j[0])
            l_apr_values.append(j[1])
        elif j[3] == 'На_здоровье':
            l_zdor_keys.append(j[0])
            l_zdor_values.append(j[1])
        elif j[3] == 'Соц_аптека':
            l_soc_keys.append(j[0])
            l_soc_values.append(j[1])
# ДЕЛАЕМ СЛОВАРИ ПО КАЖДОЙ СЕТИ ПРОИЗВОДИТЕЛЬ КАК НА САЙТЕ И ПРОИЗВОДИТЕЛЬ УНИФИЦИРОВАННЫЙ ДЛЯ ВСЕХ            
dict_change_apr_groop_goods = {k: v for k, v in zip(l_apr_keys, l_apr_values)}
dict_change_zdor_groop_goods = {k: v for k, v in zip(l_zdor_keys, l_zdor_values)}
dict_change_soc_groop_goods = {k: v for k, v in zip(l_soc_keys, l_soc_values)}
# print(dict_change_apr_groop_goods)
# print(dict_change_zdor_groop_goods)

In [19]:
# ВСТАВЛЯЕМ КОЛОНКУ ДЛЯ САЙТА АПРЕЛЬ С ОБЩИМ НАЗВАНИЕМ ПРОИЗВОДИТЕЛЯ
df_apr_prep.insert (loc= 16 , column='Manufacturer_common', value=df_apr_prep.Manufacturer)
df_apr_prep['Manufacturer_common']= df_apr_prep['Manufacturer_common'].map(dict_change_apr_groop_goods)


In [20]:
df_zdor.insert(loc = 15, column='Manufacturer_common', value=df_zdor['Производитель'])
df_zdor['Manufacturer_common'] = df_zdor['Производитель'].map(dict_change_zdor_groop_goods)

In [21]:
print(df_zdor.shape)
list_col_zdor = ['count_index', 'item_name', 'item_price', 'Срок годности', 'Manufacturer_common', 'Условия хранения', 
                 'availability_pharmacy', 'Форма выпуска', 'Действующее вещество', 'Лекарственная форма', 'По рецепту',
                 'Производитель']
rename_col_change_zdor = {'Срок годности': 'best_before_date', 'Условия хранения': 'storage_condition', 'Действие': 'action',
       'Производитель': 'Manufacturer', 'Форма выпуска': 'Form_release', 'Действующее вещество': 'Active_ingredient',
       'Лекарственная форма': 'dosage_form', 'По рецепту': 'isResipe'}
df_zdor_prep = df_zdor[list_col_zdor]
df_zdor_prep.rename(columns=rename_col_change_zdor, inplace=True) 

(1541, 31)


In [22]:
df_soc.sample(4)
df_soc_prep = df_soc[['count_index', 'item_name', 'item_price', 'Производитель', 'Страна производства', 'Форма выпуска', 'Действующее вещество', 
'Условия отпуска из аптек', 'availability_pharmacy', 'list_drugstore']]
rename_col_change_soc = {'Производитель': 'Manufacturer', 'Страна производства': 'country', 'Форма выпуска': 'Form_release', 
'Действующее вещество': 'Active_ingredient', 'Условия отпуска из аптек':'isResipe'}
df_soc_prep.rename(columns=rename_col_change_soc, inplace=True) 

df_soc_prep.insert(loc = 4, column='Manufacturer_common', value=df_soc_prep['Manufacturer'])
df_soc_prep['Manufacturer_common'] = df_soc_prep['Manufacturer'].map(dict_change_soc_groop_goods)



In [23]:
df_soc_prep.insert(2, column='name', value=df_soc_prep['item_name'].apply(lambda x: x.split('(')[0]))
df_soc_prep[['Manufacturer', 'Manufacturer_common']] = df_soc_prep[['Manufacturer', 'Manufacturer_common']].fillna('None') 
df_zdor_prep[['Manufacturer', 'Manufacturer_common']] = df_zdor_prep[['Manufacturer', 'Manufacturer_common']].fillna('None') 
df_apr_prep[['Manufacturer', 'Manufacturer_common']] = df_apr_prep[['Manufacturer', 'Manufacturer_common']].fillna('None') 
df_apr_prep.insert(2, column='name_manufact', value=df_apr_prep['name']+'_'+df_apr_prep['Manufacturer_common'])
df_zdor_prep.insert(2, column='name_manufact', value=df_zdor_prep['item_name']+'_'+df_zdor_prep['Manufacturer_common'])
df_soc_prep.insert(2, column='name_manufact', value=df_soc_prep['name']+'_'+df_soc_prep['Manufacturer_common'])
#df_soc_prep['name_manufact'] = df_soc_prep['name_manufact'].apply(lambda x: x.split('(')[0])

In [24]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import csv
from collections import Counter

In [25]:
#form apr soc - данные рейтинги можно использовать для ручной обработки наименований, т.е. в файле 
#Excel проставляют признак сопоставления одних наименований (одной сети) к другой.
name_apr = list(df_apr_prep.name_manufact.value_counts().to_dict().keys())
name_zdor = list(df_zdor_prep.name_manufact.value_counts().to_dict().keys())
name_soc = list(df_soc_prep.name_manufact.value_counts().to_dict().keys())
count_fuzzy = 0
name_iter = name_zdor
name_iter = name_soc
dict_chief = []
rating = 75
for i in name_apr[0:10]:
    count_fuzzy += 1
    for j in name_iter:
        a_sort = fuzz.token_sort_ratio(i, j)
        a_set = fuzz.token_set_ratio(i, j)
        a_w = fuzz.WRatio(i, j)
        #print(f'# {count_fuzzy}: r_sort ===> {a_sort},r_set ===> {a_set} f --> {i}, s --> {j}')
        if a_sort > rating or a_set > rating:
            manufacturer_apr = i.split('_')[-1]
            manufacturer_iter = j.split('_')[-1]
            dict_name = {'count_fuzzy': count_fuzzy, 'a_sort': a_sort, 'a_set': a_set, 
                        'a_Wratio': a_w, 'name_apr': i, 'name_soc': j, 'manuracturer_apr': manufacturer_apr,
                        'manuracturer_soc': manufacturer_iter}
            dict_chief.append(dict_name)
#             print(f'# {count_fuzzy}: r_sort ===> {a_sort},r_set ===> {a_set}, r_W ===> {a_w} f --> {i}, s --> {j}')

keys_name_iter = ['count_fuzzy', 'a_sort', 'a_set', 'a_Wratio', 'name_apr', 'name_soc', 'manuracturer_apr', 'manuracturer_soc']
with open(fr'C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\df_name_Apr_Soc.csv', 'w', encoding='utf-8', newline='\n') as csv_file:
    fieldname = keys_name_iter
    writer = csv.DictWriter(csv_file, fieldnames=fieldname)
    writer.writeheader()
    for d in dict_chief:
        writer.writerow(d)


In [26]:
#form apr zdor - данные рейтинги можно использовать для ручной обработки наименований, т.е. в файле 
#Excel проставляют признак сопоставления одних наименований (одной сети) к другой.
name_apr = list(df_apr_prep.name_manufact.value_counts().to_dict().keys())
name_zdor = list(df_zdor_prep.name_manufact.value_counts().to_dict().keys())
name_soc = list(df_soc_prep.name_manufact.value_counts().to_dict().keys())
count_fuzzy = 0
name_iter = name_apr
name_iter = name_zdor
#name_iter = name_soc
dict_chief = []
rating = 80
for i in name_apr[0:10]:
    count_fuzzy += 1
    for j in name_iter:
        a_sort = fuzz.token_sort_ratio(i, j)
        a_set = fuzz.token_set_ratio(i, j)
        a_w = fuzz.WRatio(i, j)
        #print(f'# {count_fuzzy}: r_sort ===> {a_sort},r_set ===> {a_set} f --> {i}, s --> {j}')
        if a_sort > rating or a_set > rating:
            manufacturer_apr = i.split('_')[-1]
            manufacturer_zdor = j.split('_')[-1]
            dict_name = {'count_fuzzy': count_fuzzy, 'a_sort': a_sort, 'a_set': a_set, 
                        'a_Wratio': a_w, 'name_apr': i, 'name_zdor': j, 'manufacturer_apr': manufacturer_apr,
                        'manufacturer_zdor': manufacturer_zdor}
            dict_chief.append(dict_name)
#             print(f'# {count_fuzzy}: r_sort ===> {a_sort},r_set ===> {a_set}, r_W ===> {a_w} f --> {i}, s --> {j}')

keys_name_iter = ['count_fuzzy', 'a_sort', 'a_set', 'a_Wratio', 'name_apr', 'name_zdor', 'manufacturer_apr', 'manufacturer_zdor']
with open(fr'C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\df_name_Apr_Zdor.csv', 'w', encoding='utf-8', newline='\n') as csv_file:
    fieldname = keys_name_iter
    writer = csv.DictWriter(csv_file, fieldnames=fieldname)
    writer.writeheader()
    for d in dict_chief:
        writer.writerow(d)

In [27]:
#Формируем файлы содержащие наименования по всем сетям. Просто наименования и кол-во значений
name_apr = (df_apr_prep.name.value_counts().to_dict())
name_zdor = (df_zdor_prep.item_name.value_counts().to_dict())
name_soc = (df_soc_prep.item_name.value_counts().to_dict())
# print(name_apr)
with open(fr"C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\dict_name_goods_Apr.txt", 'w') as file:
    for k, v in name_apr.items():
        file.write(f'{k}; {v}\n')
with open(fr"C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\dict_name_goods_Zdor.txt", 'w') as file:
    for k, v in name_zdor.items():
        file.write(f'{k}; {v}\n')
with open(fr"C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\dict_name_goods_Soc.txt", 'w') as file:
    for k, v in name_soc.items():
        file.write(f'{k}; {v}\n')

In [28]:
# creat matching name Number 1. Здесь разделяется наименование товара на составляюще:
# первое слово, остальная часть названия, каждое слово отдельно, название сети
file_name = {'name_apr': name_apr, 'name_zdor': name_zdor, 'name_soc': name_soc}    
name_null = []
part_all = []
with open(fr'C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\BIG_TEST.txt','w')as f: 
    f.write('name; part_null; part; last_part_list; file_name\n')
# for n in 
for k, v in file_name.items():
    for i in v:
        i = i.split('(')[:-1] if k == 'name_soc' else i.replace('(', '').replace(')', '')
        i = ''.join(i).strip()
        part = i.split(' ')
        last_part_list = part[1:]
        part_null = part[0]
        part_null = part_null.replace(' ', '').strip()
        last_part = ' '.join(part[1:])
        name_null.append(part_null)
#         print(f'name: {i}; part_null: {part_null}; part: {part}; join: {last_part}, file_name: {k}')
        with open(fr"C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\BIG_TEST.txt", 'a') as file:
            file.write(f'{i};{part_null};{part};{last_part};{k}\n')
        for j in last_part_list:
            part_all.append(j)
name_null = dict(Counter(name_null))
part_all = dict(Counter(part_all))

    

In [29]:
#создание нового столбца name_W для изменения названия
df_apr_prep.insert(1, column='name_W', value=df_apr_prep['name'])
df_zdor_prep.insert(1, column='name_W', value=df_zdor_prep['item_name'])
df_soc_prep.insert(1, column='name_W', value=df_soc_prep['name'])

df_apr_prep.name_W = df_apr_prep.name_W.str.strip()
df_soc_prep.name_W = df_soc_prep.name_W.str.strip()
df_zdor_prep.name_W = df_zdor_prep.name_W.str.strip()

In [30]:
# creat matching name Number 2
name_apr_W = list(df_apr_prep.name_W.value_counts().to_dict().keys())
name_zdor_W = list(df_zdor_prep.name_W.value_counts().to_dict().keys())
name_soc_W = list(df_soc_prep.name_W.value_counts().to_dict().keys())
file_name = {'name_apr_W': name_apr_W, 'name_zdor_W': name_zdor_W, 'name_soc_W': name_soc_W}    
name_null = []
part_all = []
with open(fr'C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\BIG_TEST2.txt','w')as f: 
    f.write('name_W; part_null; part; last_part_list; file_name\n')

for k, v in file_name.items():
    for i in v:
        #i = i.split('(')[:-1] if k == 'name_soc' else i.replace('(', '').replace(')', '')
        i = ''.join(i).strip()
        part = i.split(' ')
        last_part_list = part[1:]
        part_null = part[0]
        part_null = part_null.replace(' ', '').strip()
        last_part = ' '.join(part[1:])
        name_null.append(part_null)
#         print(f'name: {i}; part_null: {part_null}; part: {part}; join: {last_part}, file_name: {k}')
        with open(fr"C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\BIG_TEST2.txt", 'a') as file:
            file.write(f'{i.strip()};{part_null};{part};{last_part};{k}\n')
        for j in last_part_list:
            part_all.append(j)
name_null = dict(Counter(name_null))
part_all = dict(Counter(part_all))


In [31]:
# СОЗДАНИЕ СЛОВАРЯ ДЛЯ ЗАМЕНЫ НЕНУЖНЫХ ЗНАЧЕНИЙ В КОЛОНКЕ name_W
dict_= {}
with open(fr"C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\dict_replace_name.txt", 'r', encoding='utf-8') as file:
    for i in file:
#         print(i.strip().split(';')[1].replace("'", ""))
        k = i.strip().split(';')[0].replace("0", " ")
        v = i.strip().split(';')[1].replace("'", "").replace(" 0", "").replace("0", " ")#.strip()#.replace("''", "").replace("' '", " ").replace("'", "")
        dict_[k] = v
#for k, v in dict_.items():
 #   print('k =>', k, 'v =>', v,'|')

In [32]:
# ЗДЕСЬ ПРОИСХОДИТ ИЗМЕНЕНИЕ НАИМЕНОВАНИЙ В name_W !!!!!!!!!!!!!!!!!!!!!!!!!!!!!
dict_
#df_apr_total[i] = df_apr_total[i].apply(lambda x: x.replace('[]', 'Not value'))
# Максиколд рино пор. д/р-ра внутр малиновый 15г №10
df = [df_apr_prep, df_zdor_prep, df_soc_prep]
for i in range(len(df)):
    for k, v in dict_.items():
#         print(k, '|', v)
        df[i]['name_W'] = df[i]['name_W'].apply(lambda x: x.replace(k, v))

In [33]:
# ПРОВЕРКА ИЗМЕНЕНИЙ
p1 = df_zdor_prep.loc[df_zdor_prep['item_name'] == 'Тримедат форте таб.пролонг.высвоб.п.п.о. 300мг №20']
p2 = df_apr_prep.loc[df_apr_prep['name'] == 'Терафлю порошок для приготовления раствора для приема внутрь Лимон пакетики №14']
#p1
#p2

In [34]:
# Write in text file separated parts of the name thereby we're be able to replace parts of the names.  
with open(fr"C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\Part_all.txt", 'w') as file:
    for k, v in part_all.items():
        file.write(f'{k}; {v}\n')
# print(part_all)

In [35]:
# creat matching name Number 2
name_apr_W = list(df_apr_prep.name_W.value_counts().to_dict().keys())
name_zdor_W = list(df_zdor_prep.name_W.value_counts().to_dict().keys())
name_soc_W = list(df_soc_prep.name_W.value_counts().to_dict().keys())
file_name = {'name_apr_W': name_apr_W, 'name_zdor_W': name_zdor_W, 'name_soc_W': name_soc_W}    
name_null = []
part_all = []
with open(fr'C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\BIG_TEST2.txt','w')as f: 
    f.write('name_W| part_null| part| last_part_list| file_name\n')

for k, v in file_name.items():
    for i in v:
        #i = i.split('(')[:-1] if k == 'name_soc' else i.replace('(', '').replace(')', '')
        i = ''.join(i).strip()
        part = i.split(' ')
        last_part_list = part[1:]
        part_null = part[0]
        part_null = part_null.replace(' ', '').strip()
        last_part = ' '.join(part[1:])
        name_null.append(part_null)
#         print(f'name: {i}; part_null: {part_null}; part: {part}; join: {last_part}, file_name: {k}')
        with open(fr"C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\BIG_TEST2.txt", 'a') as file:
            file.write(f'{i}|{part_null}|{part}|{last_part}|{k}\n')
        for j in last_part_list:
            part_all.append(j)
name_null = dict(Counter(name_null))
part_all = dict(Counter(part_all))

In [36]:
# here we have made file that will used to make another dataframe with the colomn called FIRST_NAME
with open(fr"C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\first_name.txt", 'w') as file:
            file.write(f'name;first_name;chains\n')
with open(fr"C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\BIG_TEST2.txt", 'r') as fp:
    list_file = fp.read()
    for i in list_file.splitlines()[2:]:
        j = i.split('|')
        with open(fr"C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\first_name.txt", 'a') as file:
            file.write(f'{j[0]};{j[1]};{j[4]}\n')
        

In [37]:
# here code is trying replace some parts of the name goods in acording to file "dict_replace_name.txt"
dict_= {}
with open(fr"C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\dict_replace_name.txt", 'r', encoding='utf-8') as file:
    for i in file:
        #print(i.strip().split(';')[1].replace("'", ""))
        k = i.strip().split(';')[0]
        v = i.strip().split(';')[1].replace("'", "").replace(" 0", "")#.strip()#.replace("''", "").replace("' '", " ").replace("'", "")
        dict_[k] = v
# for k, v in dict_.items():
    #print('k =>', k, 'v =>', v,'|')

# dict_ = dict_.split(',')  
# dict_2 = {}
# for i in dict_:
#     print(i)

In [38]:
# insert new column with name and manufacturer  
df_apr_prep.insert(2, column='name_W_manufact', value=df_apr_prep['name_W']+'_'+df_apr_prep['Manufacturer_common'])
df_zdor_prep.insert(2, column='name_W_manufact', value=df_zdor_prep['name_W']+'_'+df_zdor_prep['Manufacturer_common'])
df_soc_prep.insert(2, column='name_W_manufact', value=df_soc_prep['name_W']+'_'+df_soc_prep['Manufacturer_common'])

In [39]:
# this code help to compare different items between chains 
#form apr soc NAME_W - данные рейтинги можно использовать для ручной обработки наименований, т.е. в файле 
#Excel проставляют признак сопоставления одних наименований (одной сети) к другой.
name_W_apr = list(df_apr_prep.name_W_manufact.value_counts().to_dict().keys())
name_W_zdor = list(df_zdor_prep.name_W_manufact.value_counts().to_dict().keys())
name_W_soc = list(df_soc_prep.name_W_manufact.value_counts().to_dict().keys())
count_fuzzy = 0
name_iter = name_W_zdor
name_iter = name_W_soc
dict_chief = []
rating = 75
for i in name_W_apr[0:10]:
    count_fuzzy += 1
    for j in name_iter:
        a_sort = fuzz.token_sort_ratio(i, j)
        a_set = fuzz.token_set_ratio(i, j)
        a_w = fuzz.WRatio(i, j)
        #print(f'# {count_fuzzy}: r_sort ===> {a_sort},r_set ===> {a_set} f --> {i}, s --> {j}')
        if a_sort > rating or a_set > rating:
            manufacturer_apr = i.split('_')[-1]
            manufacturer_iter = j.split('_')[-1]
            dict_name = {'count_fuzzy': count_fuzzy, 'a_sort': a_sort, 'a_set': a_set, 
                        'a_Wratio': a_w, 'name_W_apr': i, 'name_W_soc': j,'name_W_Apr_first': i.split(' ')[0], 'name_W_Soc_first': j.split(' ')[0], 
                         'manuracturer_apr': manufacturer_apr, 'manuracturer_soc': manufacturer_iter}
            dict_chief.append(dict_name)
#             print(f'# {count_fuzzy}: r_sort ===> {a_sort},r_set ===> {a_set}, r_W ===> {a_w} f --> {i}, s --> {j}')

keys_name_iter = ['count_fuzzy', 'a_sort', 'a_set', 'a_Wratio', 'name_W_apr', 'name_W_Apr_first', 'name_W_soc', 'name_W_Soc_first', 'manuracturer_apr', 'manuracturer_soc']
with open(fr'C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\df_name_Apr_Soc.csv', 'w', encoding='utf-8', newline='\n') as csv_file:
    fieldname = keys_name_iter
    writer = csv.DictWriter(csv_file, fieldnames=fieldname)
    writer.writeheader()
    for d in dict_chief:
        writer.writerow(d)


In [40]:
# FIRST NAME. здесь делаем столбец с первым именем товара.
dict_first_name = {}
with open(fr"C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\first_name.txt", 'r') as file:
    for i in file:
        k = i.strip().split(';')[0]
        v = i.strip().split(';')[1]
        dict_first_name[k] = v
df_apr_prep.insert(loc= 1 , column='first_name', value=df_apr_prep.name_W)
df_apr_prep['first_name']= df_apr_prep['first_name'].map(dict_first_name)

df_soc_prep.insert(loc= 1 , column='first_name', value=df_soc_prep.name_W)
df_soc_prep['first_name']= df_soc_prep['first_name'].map(dict_first_name)

df_zdor_prep.insert(loc= 1 , column='first_name', value=df_zdor_prep.name_W)
df_zdor_prep['first_name']= df_zdor_prep['first_name'].map(dict_first_name)

In [41]:
# здесь СЧИТАЕМ ОТКЛОНЕНИЯ ОДНОЙ БАЗЫ К ДРУГОЙ. Формируем словарь и файл с отклонениями.
# df1 = set(df_apr_prep.loc[df_apr_prep['parsing_day_x'] == '2022-12-28'].name_W_manufact.value_counts().to_dict().keys())
# df2 = set(df_apr_prep.loc[df_apr_prep['parsing_day_x'] == '2023-05-07'].name_W_manufact.value_counts().to_dict().keys())
# list_name_W_manufact = list(df1 & df2)
# print('number items FIRST & SECOND= ', len(df1), len(df2), 'total = ', len(list_name_W_manufact))
# dict_list_val_2023_05_07 = []
# for i in list_name_W_manufact:
#     dict_1 = {}
#     reserch = df_apr_prep.loc[df_apr_prep['name_W_manufact'] == i]
#     pr_2022_12_28 = reserch.loc[reserch['parsing_day_x'] == '2022-12-28'].price_withoutCard.values[0]
#     pr_2023_05_07 = reserch.loc[reserch['parsing_day_x'] == '2023-05-07'].price_withoutCard.values[0]
#     dict_1['name_W_manufact'] = i
#     dict_1['variation_2023_05_07'] = round((pr_2023_05_07/pr_2022_12_28-1), 4)
#     dict_1['pr_2023_05_07'] = pr_2023_05_07
#     dict_1['pr_2022_12_28'] = pr_2022_12_28
#     dict_list_val_2023_05_07.append(dict_1)

# list_key = list(dict_1.keys())
# print(list_key)
# with open(fr'C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\dict_list_val_2023_05_07_csv.csv', 'w', encoding='utf-8', newline='\n') as csv_file:  #encoding='utf-8',
#     fieldname = list_key
#     writer = csv.DictWriter(csv_file, fieldnames=fieldname)
#     writer.writeheader()
#     for d in dict_list_val_2023_05_07:
#         writer.writerow(d)
# # СОЗДАЕМ СЛОВАРЬ С НАИМЕНОВАНИЯМИ ТОВАРА И ОТКЛОНЕНИЯМИ В ЦЕНЕ
# file_name = fr'C:\Users\user\Scrapy_project\Archive_April\APRIL_csv\current_file\dict_list_val_2023_05_07_csv.csv'
# list_name_W_manufact = []
# list_variation_2023_05_07 = []
# with open(fr'{file_name}', 'r', encoding='utf-8') as f:
#     csv_file = csv.DictReader(f)
#     for row in csv_file:
#         list_name_W_manufact.append(row['name_W_manufact'])
#         list_variation_2023_05_07.append(row['variation_2023_05_07'])
# dict_list_var = {x: y for x, y in zip(list_name_W_manufact, list_variation_2023_05_07)}

# # ДЕЛАЕМ ВСТАВКУ СТОБЦА ДЛЯ ОТКЛОНЕНИЙ 'variation' КУДА ИЗ СЛОВАРЯ ОТКЛОНЕНИЙ ЗАПИВЫВАЕМ ОТКЛОНЕНИЯ
# df_apr_prep.insert (loc= 9 , column='variation_2023_05_07', value=df_apr_prep.name_W_manufact)
# df_apr_prep['variation_2023_05_07']= df_apr_prep['variation_2023_05_07'].map(dict_list_var)
# df_apr_prep.variation_2023_05_07 = df_apr_prep.variation_2023_05_07.fillna(0.0)
# # ДЕЛАЕМ СТОЛБЕЦ С ДНЁМ ПАРСИНГА
# df_apr_prep.insert (loc= 10 , column='pars_day', value=df_apr_prep.count_index)
# df_apr_prep.pars_day = df_apr_prep.pars_day.apply(lambda x: str(x.split('_')[1]))

In [42]:
df_apr_prep.insert (loc= 1 , column='pars_day', value=df_apr_prep.count_index)
df_apr_prep.pars_day = df_apr_prep.pars_day.apply(lambda x: str(x.split('_')[1]))

df_soc_prep.insert (loc= 1 , column='pars_day', value=df_soc_prep.count_index)
df_soc_prep.pars_day = df_soc_prep.pars_day.apply(lambda x: str(x.split('_')[1]))

df_zdor_prep.insert (loc= 1 , column='pars_day', value=df_zdor_prep.count_index)
df_zdor_prep.pars_day = df_zdor_prep.pars_day.apply(lambda x: str(x.split('_')[1]))

In [43]:
# dict_apr_pharmtherapy = {}
# pharm = df_apr_prep[['first_name', 'Pharmacotherapeutic_group']]
# pharm.Pharmacotherapeutic_group.value_counts().count()
# pharm.groupby(['first_name', 'Pharmacotherapeutic_group']).count()

In [44]:
# создаем одинаковые датафреймы 
col_apr = ['count_index', 'pars_day', 'first_name', 'name_W', 'name_W_manufact', 'name', 'name_manufact', 'price_withoutCard', 'Manufacturer', 'Manufacturer_common']
col_soc = ['count_index', 'pars_day', 'first_name', 'name_W', 'name_W_manufact', 'item_name', 'name_manufact', 'item_price', 'Manufacturer', 'Manufacturer_common']
col_zdor= ['count_index', 'pars_day', 'first_name', 'name_W', 'name_W_manufact', 'item_name', 'name_manufact', 'item_price', 'Manufacturer', 'Manufacturer_common']
rename_col2 = {'price_withoutCard': 'item_price', 'name': 'item_name'} 
df_apr_prep_copy = df_apr_prep
df_apr_prep = df_apr_prep[col_apr]
df_apr_prep.rename(columns=rename_col2, inplace=True)
df_apr_prep.sample(2)
df_soc_prep = df_soc_prep[col_soc]
df_zdor_prep = df_zdor_prep[col_zdor]

In [45]:
# смотрим нулевые значения цены в датафреймах
dict_df = {'df_apr_prep': df_apr_prep, 'df_soc_prep': df_soc_prep, 'df_zdor_prep': df_zdor_prep}
df_apr_prep.loc[df_apr_prep['item_price'] == 0].shape
l_df = list(dict_df.keys())
for k, v in dict_df.items():
    nul_df = v.loc[v['item_price'] == 0].shape[0]
    print(k, ' = ', nul_df)
df_apr_prep = df_apr_prep.loc[df_apr_prep['item_price'] != 0]
df_soc_prep = df_soc_prep.loc[df_soc_prep['item_price'] != 0]
df_zdor_prep = df_zdor_prep.loc[df_zdor_prep['item_price'] != 0]

ddf1 = df_apr_prep.loc[df_apr_prep['item_price'] == 0].shape
ddf2 = df_soc_prep.loc[df_soc_prep['item_price'] == 0].shape
ddf3 = df_zdor_prep.loc[df_zdor_prep['item_price'] == 0].shape

print(l_df[0], ddf1, l_df[1], ddf2, l_df[2], ddf3)

df_apr_prep  =  0
df_soc_prep  =  0
df_zdor_prep  =  0
df_apr_prep (0, 10) df_soc_prep (0, 10) df_zdor_prep (0, 10)


In [46]:
# Словарь для организации файлов
#dict_num_pars_day
#df_apr_prep = pd.concat([df_conect_2022_12_28, df_conect_2023_05_07], ignore_index=True)
#df_apr.sample(2)

In [47]:
df_soc_prep.sample(2)

,count_index,pars_day,first_name,name_W,name_W_manufact,item_name,name_manufact,item_price,Manufacturer,Manufacturer_common
101,102_2023-06-01,2023-06-01,Рамиприл,Рамиприл таблетки 10мг №30,Рамиприл таблетки 10мг №30_Синтез,Рамиприл таб 10мг N30 (Алиум),Рамиприл таб 10мг N30 _Синтез,178.00,Синтез,Синтез
1643,1644_2023-06-01,2023-06-01,Викаир,Викаир таблетки №20,Викаир таблетки №20_Фармстандарт,Викаир таб N20 (Фармстандарт МНН),Викаир таб N20 _Фармстандарт,62.00,Фармстандарт,Фармстандарт


In [48]:
df_zdor_prep.sample(2)


,count_index,pars_day,first_name,name_W,name_W_manufact,item_name,name_manufact,item_price,Manufacturer,Manufacturer_common
796,797_2023-05-30,2023-05-30,Антигриппин,Антигриппин таблетки шип. со вкусом и ароматом...,Антигриппин таблетки шип. со вкусом и ароматом...,Антигриппин таб. шип. со вкусом и ароматом мал...,Антигриппин таб. шип. со вкусом и ароматом мал...,451.00,Натур Продукт Фарма Сп. з о.о.,Натур Продукт
1395,1396_2023-05-30,2023-05-30,Тизанидин-тева,Тизанидин-тева таблетки 4мг №30,Тизанидин-тева таблетки 4мг №30_Тева,Тизанидин-тева таб. 4мг №30,Тизанидин-тева таб. 4мг №30_Тева,238.00,Тева Фармацевтические Предприятия Лтд/пр.Тева ...,Тева


In [49]:
df_apr_prep.sample(2)

,count_index,pars_day,first_name,name_W,name_W_manufact,item_name,name_manufact,item_price,Manufacturer,Manufacturer_common
1821,2163_2023-05-07,2023-05-07,Арифам,"Арифам МВ таблетки 10мг+1,5мг №30","Арифам МВ таблетки 10мг+1,5мг №30_Сервье","Арифам МВ таблетки 10мг+1,5мг №30","Арифам МВ таблетки 10мг+1,5мг №30_Сервье",657.00,Servier Ind. Ltd./ Serdix/ Сервье/ Сердикс,Сервье
1359,1548_2023-05-07,2023-05-07,Пантопразол,Пантопразол таблетки 20мг №56,Пантопразол таблетки 20мг №56_Озон,Пантопразол таблетки 20мг №56,Пантопразол таблетки 20мг №56_Озон,361.00,"ООО ""Озон""",Озон


In [50]:
df_apr_prep.sample(2)

,count_index,pars_day,first_name,name_W,name_W_manufact,item_name,name_manufact,item_price,Manufacturer,Manufacturer_common
1339,1528_2023-05-07,2023-05-07,Ренни,Ренни таблетки жевательные Апельсин №24,Ренни таблетки жевательные Апельсин №24_Байер,Ренни таблетки жевательные Апельсин №24,Ренни таблетки жевательные Апельсин №24_Байер,534.00,"Bayer Schering Pharma, AG/ Байер Шеринг",Байер
433,491_2023-05-07,2023-05-07,Амелотекс,"Амелотекс таблетки 7,5мг №20","Амелотекс таблетки 7,5мг №20_Сотекс","Амелотекс таблетки 7,5мг №20","Амелотекс таблетки 7,5мг №20_Сотекс",293.00,"ЗАО ""Сотекс Фармфирма""",Сотекс


In [51]:
sheets = {'df_apr_prep': df_apr_prep, 'df_soc_prep': df_soc_prep, 'df_zdor_prep': df_zdor_prep}
writer = pd.ExcelWriter(fr'C:\Users\user\Scrapy_project\Pharmacy_base\all_df.xlsx', engine='xlsxwriter')
for sheet_name in sheets.keys():
    sheets[sheet_name].to_excel(writer, sheet_name=sheet_name)
writer.save()

In [52]:
dict_apr_pharmtherapy = {}
pharm = df_apr_prep_copy[['first_name', 'Pharmacotherapeutic_group']]
pharm.Pharmacotherapeutic_group.value_counts().count()
group_pharmtherapy = pharm.groupby(['first_name', 'Pharmacotherapeutic_group']).count()
group_pharmtherapy.to_excel(fr'C:\Users\user\Scrapy_project\Pharmacy_base\group_pharmtherapy.xlsx', encoding='utf-8')
#df_apr_prep_copy.sample(3)

In [53]:
#df_soc_prep.to_excel(fr'C:\Users\user\Scrapy_project\Pharmacy_base\df_all\{}.xlsx', encoding='utf-8')')
#list_write_file = 
# Словарь для организации файлов
#print(dict_num_pars_day)
for k, v in dict_df.items():
    pars_day = v.pars_day.value_counts().to_dict().keys()
    pars_day = list(pars_day)[0]
    count_pars = dict_num_pars_day[pars_day][0]
    chain_pharm = dict_num_pars_day[pars_day][1]
    print(k, pars_day, count_pars, chain_pharm)
df_apr_prep['count_pars']= dict_num_pars_day[list(df_apr_prep.pars_day.value_counts().to_dict().keys())[0]][0]
df_soc_prep['count_pars']= dict_num_pars_day[list(df_soc_prep.pars_day.value_counts().to_dict().keys())[0]][0]
df_zdor_prep['count_pars']= dict_num_pars_day[list(df_zdor_prep.pars_day.value_counts().to_dict().keys())[0]][0]

df_apr_prep['chain_pharm']= dict_num_pars_day[list(df_apr_prep.pars_day.value_counts().to_dict().keys())[0]][1]
df_soc_prep['chain_pharm']= dict_num_pars_day[list(df_soc_prep.pars_day.value_counts().to_dict().keys())[0]][1]
df_zdor_prep['chain_pharm']= dict_num_pars_day[list(df_zdor_prep.pars_day.value_counts().to_dict().keys())[0]][1]

df_apr_prep 2023-05-07 2 apr
df_soc_prep 2023-06-01 2 soc
df_zdor_prep 2023-05-30 2 zdor


In [54]:
df_apr_prep.sample(3)
#df_soc_prep.sample(3)
#df_zdor_prep.sample(3)
dict_num_pars_day
print(dict_num_pars_day)
for k, v in dict_df.items():
    pars_day = v.pars_day.value_counts().to_dict().keys()
    pars_day = list(pars_day)[0]
    count_pars = dict_num_pars_day[pars_day][0]
    chain_pharm = dict_num_pars_day[pars_day][1]
    print(k, pars_day, count_pars, chain_pharm)

{'2022-12-28': [1, 'apr'], '2023-05-07': [2, 'apr'], '2023-01-18': [1, 'soc'], '2023-06-01': [2, 'soc'], '2023-01-16': [1, 'zdor'], '2023-05-30': [2, 'zdor']}
df_apr_prep 2023-05-07 2 apr
df_soc_prep 2023-06-01 2 soc
df_zdor_prep 2023-05-30 2 zdor


In [55]:
ddd = df_apr_prep.shape[0]
ddd1 = df_zdor_prep.shape[0]
ddd2 = df_soc_prep.shape[0]
print(ddd, ddd1, ddd2)

1879 1541 1753


In [56]:
#сделать файлы с номерным индексок в названии
dict_num_pars_day
dict_num_pars_day[parsing_day_april][0]

2

In [57]:
df_all_pharm = pd.concat([df_apr_prep, df_soc_prep], ignore_index=True)
df_all_pharm = pd.concat([df_all_pharm, df_zdor_prep], ignore_index=True)
df_all_pharm.to_excel(fr'C:\Users\user\Scrapy_project\Pharmacy_base\df_all\df_all_pharm_{dict_num_pars_day[parsing_day_april][0]}.xlsx', encoding='utf-8')
df_apr_prep.to_excel(fr'C:\Users\user\Scrapy_project\Pharmacy_base\df_all\df_apr_prep_{dict_num_pars_day[parsing_day_april][0]}.xlsx', encoding='utf-8')
df_soc_prep.to_excel(fr'C:\Users\user\Scrapy_project\Pharmacy_base\df_all\df_soc_prep_{dict_num_pars_day[parsing_day_soc][0]}.xlsx', encoding='utf-8')
df_zdor_prep.to_excel(fr'C:\Users\user\Scrapy_project\Pharmacy_base\df_all\df_zdor_prep_{dict_num_pars_day[parsing_day_zdor][0]}.xlsx', encoding='utf-8')

In [58]:
import os
dirname_apr = fr'C:\Users\user\Scrapy_project\Pharmacy_base\df_all'
path = fr'C:\Users\user\Scrapy_project\Pharmacy_base\df_all'
files = os.listdir(dirname_apr)
apr = []
soc = []
zdor = []
all_files = []
for i in files:
    if 'df_apr' in i:
        apr.append(i)
        all_files.append(i)
    elif 'df_soc' in i:
        soc.append(i)
        all_files.append(i)
    elif 'df_zdor' in i:
        zdor.append(i)
        all_files.append(i)
apr.sort()
soc.sort()
zdor.sort()
print('apr ', apr, 'soc ', soc, 'zdor ', zdor, sep='\n')
print(all_files)


apr 
['df_apr_prep_1.xlsx', 'df_apr_prep_2.xlsx']
soc 
['df_soc_prep_1.xlsx', 'df_soc_prep_2.xlsx']
zdor 
['df_zdor_prep_1.xlsx', 'df_zdor_prep_2.xlsx']
['df_apr_prep_1.xlsx', 'df_apr_prep_2.xlsx', 'df_soc_prep_1.xlsx', 'df_soc_prep_2.xlsx', 'df_zdor_prep_1.xlsx', 'df_zdor_prep_2.xlsx']


In [59]:
count_ = 1
for i in all_files:
    f_path = path+f'\\'+i
    fl = pd.read_excel(f_path)
    if count_ >= 2:
        df_x = pd.concat([df_x, fl], ignore_index=True)
    else:
        df_x = fl
        print(count_)
    count_ += 1
    print(i, f_path)

1
df_apr_prep_1.xlsx C:\Users\user\Scrapy_project\Pharmacy_base\df_all\df_apr_prep_1.xlsx
df_apr_prep_2.xlsx C:\Users\user\Scrapy_project\Pharmacy_base\df_all\df_apr_prep_2.xlsx
df_soc_prep_1.xlsx C:\Users\user\Scrapy_project\Pharmacy_base\df_all\df_soc_prep_1.xlsx
df_soc_prep_2.xlsx C:\Users\user\Scrapy_project\Pharmacy_base\df_all\df_soc_prep_2.xlsx
df_zdor_prep_1.xlsx C:\Users\user\Scrapy_project\Pharmacy_base\df_all\df_zdor_prep_1.xlsx
df_zdor_prep_2.xlsx C:\Users\user\Scrapy_project\Pharmacy_base\df_all\df_zdor_prep_2.xlsx


In [60]:
df_x.sample(3)
df_x.Manufacturer.value_counts()

None                                    348
KRKA d.d./ КРКА                         237
КРКА                                    195
ООО "Озон"                              192
Озон                                    141
                                       ... 
Тверская фармфабрика                      1
Noventis S.R.O./ Новентис                 1
Англо-Френч Драгс энд Индастриз           1
Nabros Pharma/ Наброс Фарма               1
Ксантис Фарма Лимитед/Алси Фарма ЗАО      1
Name: Manufacturer, Length: 1197, dtype: int64

In [61]:
shape_soc = df_soc_prep.loc[df_soc_prep['Manufacturer'] == 'None'].shape[0]
shape_zdor = df_zdor_prep.loc[df_zdor_prep['Manufacturer'] == 'None'].shape[0]
#df_soc.sample(3)

df_x.loc[df_x['Manufacturer'] == 'None'][['item_name', 'chain_pharm']].to_excel(fr'C:\Users\user\Scrapy_project\Pharmacy_base\none_group_manufact.xlsx')
list_none = df_x.loc[df_x['Manufacturer'] == 'None']['item_name'].to_list()
# проверка производителей в базе soc
df_soc.loc[df_soc['item_name'] == 'Калмирекс табс таб п/пл/о 150мг N30 (Сотекс)']['Производитель'].to_list()[0]#.count()
count_none_soc = 0
for i in list_none:
    x = df_soc.loc[df_soc['item_name'] == i]['Производитель'].to_list()
    if x:
        zzzz = 0
#         print('soc', x[0])
    else:
#         print(x)
        count_none_soc += 1
count_none_zdor = 0
for i in list_none:
    x = df_zdor.loc[df_zdor['item_name'] == i]['Производитель'].to_list()
    if x:
        zzzz = 0
#         print(x[0])
    else:
#         print(x)
        count_none_zdor += 1

print('shape soc', shape_soc)
print('shape zdor', shape_zdor)
print()
print('count_none_soc', count_none_soc)
print('count_none_zdor', count_none_zdor)

shape soc 57
shape zdor 119

count_none_soc 252
count_none_zdor 123


In [83]:
df_x.loc[(df_x['count_pars']==1) & (df_x['count_pars'] != 2)]
df_x[df_x.duplicated(['count_pars', 'Manufacturer', 'item_name', 'chain_pharm'])]#.value_counts()
size_dup = df_x.groupby(['count_pars', 'item_name', 'Manufacturer', 'chain_pharm'], as_index=False).size()
size_dup.loc[size_dup['size'] == 2]

,count_pars,item_name,Manufacturer,chain_pharm,size
3506,1,Престанс таблетки 10мг+5мг №30,Servier Ind. Ltd./ Serdix/ Сервье/ Сердикс,apr,2
3507,1,Престанс таблетки 5мг+10мг №30,Servier Ind. Ltd./ Serdix/ Сервье/ Сердикс,apr,2
8675,2,Престанс таблетки 10мг+10мг №30,Servier Ind. Ltd./ Serdix/ Сервье/ Сердикс,apr,2
8676,2,Престанс таблетки 10мг+5мг №30,Servier Ind. Ltd./ Serdix/ Сервье/ Сердикс,apr,2
8677,2,Престанс таблетки 5мг+10мг №30,Servier Ind. Ltd./ Serdix/ Сервье/ Сердикс,apr,2
8678,2,Престанс таблетки 5мг+5мг №30,Servier Ind. Ltd./ Serdix/ Сервье/ Сердикс,apr,2


In [84]:
df_x.loc[(df_x['item_name']=='Престанс таблетки 5мг+10мг №30') & (df_x['count_pars'] == 1) & (df_x['chain_pharm'] == 'apr')]

,Unnamed: 0,count_index,pars_day,first_name,name_W,name_W_manufact,item_name,name_manufact,item_price,Manufacturer,Manufacturer_common,count_pars,chain_pharm
1753,1753,2267_2022-12-28,2022-12-28,Престанс,Престанс таблетки 5мг+10мг №30,Престанс таблетки 5мг+10мг №30_Сервье,Престанс таблетки 5мг+10мг №30,Престанс таблетки 5мг+10мг №30_Сервье,1081,Servier Ind. Ltd./ Serdix/ Сервье/ Сердикс,Сервье,1,apr
1821,1821,2335_2022-12-28,2022-12-28,Престанс,Престанс таблетки 5мг+10мг №30,Престанс таблетки 5мг+10мг №30_Сервье,Престанс таблетки 5мг+10мг №30,Престанс таблетки 5мг+10мг №30_Сервье,1083,Servier Ind. Ltd./ Serdix/ Сервье/ Сердикс,Сервье,1,apr


In [144]:
list_chain = ['apr', 'soc', 'zdor']
max_ = list(df_x.count_pars.value_counts().to_dict().keys())
max_.sort()
max(max_)
# for i in list_chain:
#     for j in max_:
        
#for i in max_:
#len(set(
len(set(df_x.loc[(df_x['count_pars'] == 1) & (df_x['chain_pharm'] == 'apr')].name_W_manufact.value_counts().to_dict().keys()))

1926

In [ ]:
xy = [1, 2, 3, 4]


In [63]:
df_x.sample(3)

,Unnamed: 0,count_index,pars_day,first_name,name_W,name_W_manufact,item_name,name_manufact,item_price,Manufacturer,Manufacturer_common,count_pars,chain_pharm
3339,1411,1631_2023-05-07,2023-05-07,Долгит,Долгит крем 5% 50г,Долгит крем 5% 50г_Долоргет,Долгит крем 5% 50г,Долгит крем 5% 50г_Долоргет,149,Dolorgiet GmbH/ Долоргет,Долоргет,2,apr
9937,1199,1200_2023-05-30,2023-05-30,Энтерол,Энтерол капс. 250мг №30 (блистер),Энтерол капс. 250мг №30 (блистер)_Биокодекс,Энтерол капс. 250мг №30 (блистер),Энтерол капс. 250мг №30 (блистер)_Биокодекс,914,Биокодекс С.А.,Биокодекс,2,zdor
5038,1231,1232_2023-01-18,2023-01-18,Мотилиум,Мотилиум таблетки 10мг №30,Мотилиум таблетки 10мг №30_Янссен Силаг,Мотилиум таб 10мг N30 (Джонсон),Мотилиум таб 10мг N30 _Янссен Силаг,861.0,Янссен-Силаг,Янссен Силаг,1,soc


In [2]:

!python --version

Python 3.9.15
